# Hamoye Week 4 July 2021 Internship Assignment

In this assignment, we will build artificial intelligence algorithms to label satellite image chips with different atmospheric conditions and the different classes of land cover/land use from Kaggle. The algorithms from this project will enable us to understand where, how and why deforestation happens.
The dataset used in this assignment can be downloaded [here](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data).

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-10-19 20:18:49.605571: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-19 20:18:49.605700: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
print(tf.__version__)

2.4.1


In [4]:
# the following is for me to split the dataset.
train = pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')
test = pd.read_csv('/kaggle/input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv')

In [5]:
labels = train['tags'].apply(lambda x: x.split(' '))

In [6]:
label_list=[]
for i in labels:
    for j in i:
        if j not in label_list:
            label_list.append(j)

In [7]:
x_train = []
x_test = []
y_train = []

In [8]:
label_map = {l: i for i, l in enumerate(label_list)}

In [9]:
from tqdm import tqdm
import cv2

In [10]:
# reading the images for model training
for f, tags in tqdm(train.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)

100%|██████████| 40479/40479 [03:28<00:00, 194.09it/s]


In [11]:
for f, tags in tqdm(test.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    x_test.append(cv2.resize(img,(64, 64)))

 66%|██████▋   | 40669/61191 [04:24<02:13, 153.84it/s]


error: OpenCV(4.5.3) /tmp/pip-req-build-fvfwe_ss/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [1]:
y_train = np.array(y_train)
x_train = np.array(x_train, np.float32)/255.0
x_test  = np.array(x_test, np.float32)/255.0

NameError: name 'np' is not defined

In [13]:
print(x_train.shape)
#print(x_test.shape)
print(y_train.shape)

(40479, 64, 64, 3)
(40479, 17)


In [14]:
# Importing needed libraries
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
np.random.seed(1671)

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [15]:
# building the model, 
model = Sequential()
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(17))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0

2021-10-19 20:27:44.496868: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-19 20:27:44.499902: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-19 20:27:44.499946: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-19 20:27:44.499983: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (175c95a21320): /proc/driver/nvidia/version does not exist
2021-10-19 20:27:44.501027: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [17]:
OPTIMIZER = Adam(lr=0.01)  #Adam(lr=0.01), RMSprop .8624, sgd
model.compile(loss='binary_crossentropy', optimizer=OPTIMIZER , metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=1, validation_split=0.25)
# I noticed that SGD and RMSprop have the same val_accuracy 0.8624 when batch_size = 64, epochs=3, verbose=1 and validation_split = 0.25

2021-10-19 20:36:16.705311: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1492205568 exceeds 10% of free system memory.


Epoch 1/3
475/475 [==============================] - 439s 923ms/step - loss: 0.2585 - accuracy: 0.8257 - val_loss: 0.2243 - val_accuracy: 0.8770
Epoch 2/3
475/475 [==============================] - 435s 916ms/step - loss: 0.2157 - accuracy: 0.8642 - val_loss: 0.1829 - val_accuracy: 0.8915
Epoch 3/3
475/475 [==============================] - 436s 919ms/step - loss: 0.1892 - accuracy: 0.8361 - val_loss: 0.1787 - val_accuracy: 0.8885


In [18]:
import gc #to clear the RAM
gc.collect()

4008

In [19]:
y_test=[]
p_test = model.predict(x_test)
y_test.append(p_test)

2021-10-19 20:58:09.525448: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1998962688 exceeds 10% of free system memory.


In [20]:
result = np.array(y_test[0])
result = pd.DataFrame(result, columns = label_list)
result

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,1.005881e-05,0.707933,0.000107,2.914343e-01,0.000257,1.346380e-05,7.595703e-05,7.017043e-05,3.620305e-06,1.737507e-06,0.000014,1.678059e-07,5.616309e-06,5.431223e-08,3.718037e-05,3.136872e-05,5.760841e-06
1,6.000078e-06,0.661120,0.000053,3.385188e-01,0.000145,8.156512e-06,4.357786e-05,3.681991e-05,2.447827e-06,1.236853e-06,0.000007,9.422494e-08,2.987647e-06,2.665651e-08,2.919121e-05,2.170099e-05,3.948247e-06
2,4.185701e-12,0.997841,0.000002,1.238508e-07,0.000004,1.227662e-07,8.836696e-07,3.391213e-07,2.367737e-11,1.456318e-11,0.002151,4.753395e-10,1.343300e-08,4.512918e-08,1.096744e-11,2.215257e-10,3.086410e-14
3,2.740634e-05,0.948816,0.001109,4.725643e-02,0.000935,1.048991e-04,4.731054e-04,7.931934e-04,2.775170e-05,3.052776e-06,0.000227,1.942621e-06,3.767324e-05,7.314068e-07,6.637268e-05,1.026512e-04,1.837427e-05
4,2.007879e-04,0.862201,0.028310,8.713070e-03,0.011378,6.314241e-03,1.724366e-02,3.128571e-03,2.888935e-05,3.817301e-04,0.059527,4.449253e-04,5.333856e-04,1.552568e-03,1.224038e-05,2.594275e-05,4.502231e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40664,6.631565e-06,0.678061,0.000059,3.215459e-01,0.000158,8.829830e-06,4.744886e-05,4.026467e-05,2.570568e-06,1.351567e-06,0.000008,1.059812e-07,3.289864e-06,3.057278e-08,3.017713e-05,2.288663e-05,4.104502e-06
40665,8.538811e-05,0.950173,0.008085,8.915849e-03,0.006521,1.289492e-03,5.115471e-03,2.936020e-03,5.978632e-05,6.090169e-05,0.015911,4.223126e-05,3.262145e-04,2.302890e-04,3.444636e-05,1.950870e-04,1.919675e-05
40666,6.585827e-03,0.722575,0.077081,1.098910e-01,0.019317,6.819007e-03,3.072703e-02,1.231063e-02,3.097333e-04,1.420899e-03,0.009805,2.808960e-04,1.865998e-03,1.749190e-04,1.579229e-04,5.066163e-04,1.707590e-04
40667,3.954406e-11,0.994370,0.000008,3.545667e-07,0.000015,1.178923e-06,3.083817e-06,8.309848e-07,9.620839e-11,2.069547e-10,0.005601,6.786137e-09,3.491900e-08,9.864609e-08,6.903167e-11,1.011509e-10,1.074651e-13


In [21]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))
    


100%|██████████| 40669/40669 [01:04<00:00, 626.97it/s]


In [25]:
test['tags'][61191] = preds
test.to_csv('sec_submission.csv', index=False)